This is a notebook to consolidate proteingroups that share the same ENSG sets. For more detailed explanation, please read https://czbiohub.atlassian.net/l/cp/vS2GnF2b

In [3]:
import sys

# The notebook requires scripts from OpenCell (https://github.com/czbiohub/opencell/)
# designate where the OpenCell repo is cloned, and import modules
sys.path.append('../../opencell/')

# appending path for Pyseus
sys.path.append('../')

from opencell.database import ms_utils, utils

from pyseus import basic_processing as ip

import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker


In [4]:
# ensembl uniprot table is used to map protein IDs to ENSG ids
# code to query ensembl_uniprot from the OC db
url = utils.url_from_credentials('../data/OC_database/db-credentials-cap.json')
engine = sqlalchemy.create_engine(url)
engine.connect()

# initiate session
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

# retrieve the ensembl-uniprot association table
ensembl_uniprot = pd.read_sql('select * from ensembl_uniprot_association', engine)

In [79]:
pd.read_sql('select * from mass_spec_protein_group limit 10', engine)

,id,gene_names,uniprot_ids,date_created,manual_gene_name
0,7a1e367563ade1e1a962052022d30b7c6941e859a639b6...,"[hCG_39893, GRAMD1B]","[A0A024R3M2, A0A0J9YXF6, A0A0J9YXZ1, A0A0J9YY7...",2020-04-13 22:33:27.313001+00:00,None
1,ff8cc36c59fc628cdf2fb7b929611f6733b7a4bdb87bb0...,[DNPEP],"[A0A024R442, B9ZVU2, C9JBE1, E5RJ35, E7EMB6, E...",2020-04-13 22:33:27.315791+00:00,None
2,283c53f3f5dbc8de867faa6300e116abcbd4bcb87cfc4e...,[HDLBP],"[A0A024R4E5, C9J5E5, C9JBS3, C9JEJ8, C9JES8, C...",2020-04-13 22:33:27.317712+00:00,None
3,fd86ebee1175a0960c9e20b08f8dcc4af5e2d871766f3f...,[SERPINA1],"[A0A024R6I7, A0A0G2JRN3, P01009, P01009-2]",2020-04-13 22:33:27.321574+00:00,None
4,5dffc01e132ef1bbf454295e001ea39dfcdd63688d8e2c...,"[BAG6, BAT3]","[A0A024RCR6, A0A0G2JJM1, A0A0G2JJR8, A0A0G2JK2...",2020-04-13 22:33:27.329079+00:00,None
5,2210ef82e3bbdbd6c06de29501ed9a213780ec86acf19d...,[ATP11C],"[A0A067XG54, H7C0E8, Q8NB49, Q8NB49-2, Q8NB49-...",2020-04-13 22:33:27.331009+00:00,None
6,6f8355d59dc8d2e357099947df8662c00398469c52835a...,[BCORL1],"[A0A075B6Q3, H7C4B2, Q5H9F3, Q5H9F3-2, Q5H9F3-3]",2020-04-13 22:33:27.334772+00:00,None
7,22c3139a9e9ca05bfc008c522890da40523068589a2aa0...,[SYNM],"[A0A075B7B1, H0YL34, O15061, O15061-2]",2020-04-13 22:33:27.341027+00:00,None
8,75e3d97722897ecd10976e00fc934d15760a75b6ce5d68...,[HELLS],"[A0A087WSW7, A0A0B4J1V9, F6XU50, Q9NRZ9, Q9NRZ...",2020-04-13 22:33:27.350357+00:00,None
9,08ad4c5cbb5eb95cfa380a5b6ca171577ef19709c31ede...,[ACTN3],"[A0A087WSZ2, Q08043]",2020-04-13 22:33:27.354221+00:00,None


#### An example of the current state of the protein group organization in OpenCell v1.0 
The explanation of how protein groups were consolidated are explained in the Confluence document linked at the top.
This code bit shows that the implementation was faulty, that there are still duplicate proteingroups with identical gene names in the mass_spec_hit upload. The probable reasons why this is so is also included in the document. 


In [64]:
# Sql query to retrieve protein group ids, uniprot ids, and gene names. 
# Pulling from hit table to retrieve only protein groups that are marked significant
# And joining with protein_group table for the actual protein group information

primary_ids = pd.read_sql('select distinct mass_spec_protein_group.id AS pg_id, hit.pulldown_id,\
    mass_spec_protein_group.uniprot_ids, mass_spec_protein_group.gene_names  \
    from mass_spec_hit hit\
    inner join mass_spec_protein_group on hit.protein_group_id\
    =mass_spec_protein_group.id\
    where hit.is_significant_hit = TRUE or hit.is_minor_hit = True'\
   , engine)

In [77]:
# The next steps are pandas operations to identify proteingroups that should have been consolidated under gene names, but did not

duplicates = primary_ids.drop_duplicates(subset=['pg_id']).copy()
# lists cannot be used in dropna or duplicated function, so converting to string
duplicates['gene_names'] = duplicates['gene_names'].apply(str)
duplicates[duplicates.duplicated(subset='gene_names', keep=False)].sort_values('gene_names')[:10]

,pg_id,pulldown_id,uniprot_ids,gene_names
4466,24308a343ada832b9d2a8157af758d4f705e336853b6e0...,1802,[O95573],['ACSL3']
16780,8b047a0f0823e80de7042b4fbb475d8b5488b388c8f818...,3400,"[F5GWH2, O95573]",['ACSL3']
26110,d6f38f42509a8f67b21d0742b3d8a5fc73be8141522da2...,2133,"[A0A087WX84, A0A0A0MRE9, A0A0A0MRF6, Q99996, Q...",['AKAP9']
13011,6dc19a6ccaee5214cd455110340953ebdf1e63f0bcac68...,3389,"[A0A087WX84, A0A0A0MRE9, A0A0A0MRF6, A0A2R8Y59...",['AKAP9']
29195,f11d31a205af3ede20ca5b711a127c29fd0bd1c7494ea6...,3391,"[I3L1M4, I3L2W1, J3KTD9, J3QRD1, J3QS00, P5164...",['ALDH3A2']
24385,c9034a3930c21b652d3fa5f22eb340da5705acde728fbc...,3474,"[P51648, P51648-2]",['ALDH3A2']
15171,7cbff28cc685adcb0accd3763a7630f8f8e9327fcf1b8f...,1799,"[A8MYB8, C9JKT2, C9JMC5, E9PJV0, E9PNN6, I3L1M...",['ALDH3A2']
10244,56206f5a4d60caaa25604c93a6ba1f795a61582c733db7...,3474,"[Q9H6U8, Q9H6U8-2, Q9H6U8-3, Q9H6U8-4]",['ALG9']
1627,0c7a304bdb962cb8035e50fea67fe78ec4bbebd05523fa...,1855,"[A0A087WTZ3, A0A087WV58, A0A087WVC0, A0A087WZ6...",['ALG9']
11467,5f1b53c7fbf129698282aee5e912cfe11a0a505e2d8b71...,2311,"[F5H1D4, F5H6J0, Q86XL3, Q86XL3-2, Q86XL3-3]",['ANKLE2']


### Below are proposed, not implemented, steps to consolidate protein groups

While these steps can be wrapped into one clean function, I wanted to show individual steps so that the logic can be implemented in the future

In [15]:
# we will use output from OC_Plate_22-25_MBR, renamed table
# Simple filtering to remove irrelevant rows
pgroups = pd.read_csv('../data/OC_Plate_22-25_MBR/proteinGroups_renamed.txt', sep='\t',
    low_memory=False, index_col=0)

# basic filtering 
process = ip.RawTables(proteingroup=pgroups, file_designated=True, intensity_type='LFQ intensity')
process.filter_table()
filtered_table = process.filtered_table.copy()

# for protein group consolidation, we only need the column of protein IDs for now
protein_ids = filtered_table[['Protein IDs']].copy()

Filtered 169 of 3829 rows. Now 3660 rows.


In [35]:
# first, hash the Protein IDs
protein_ids['hashed_id'] = protein_ids['Protein IDs'].apply(
    lambda x: ms_utils.create_protein_group_id(x)[0])

# then, using the ensembl_uniprot table, associate all the ENSG IDs (sorted alphabetically to the Protein IDs
# refer to the proteingroups_to_ensgs function at the bottom of this section
protein_ids['ensg_ids'] = protein_ids['Protein IDs'].apply(lambda x: proteingroups_to_ensgs(x, ensembl_uniprot))

In [60]:
# there will be some proteingroups that will have no ENSG associated
# Drop these entries for consolidation sake, but keep track of these 
pids_dropna = protein_ids.dropna(subset='ensg_ids').copy()
pids_isna = protein_ids[protein_ids['ensg_ids'].isna()].copy()

# you can see that even in this small dataset, there are 93 proteingroups that point to the same ENSGs
pids_dropna.shape[0] - pids_dropna['ensg_ids'].nunique()

93

In [65]:
# here we are grouping proteingroups with the identical ensg_ids together with a group function
# think of it as a reverse of 'explode' function
ensg_grouped = pd.DataFrame(protein_ids.groupby('ensg_ids')['hashed_id'].apply(
    lambda x: sorted(list(x)))).reset_index(drop=False)

# since the list of hashed ids are sorted, pick the first one as a primary id 
ensg_grouped['primary_id'] = ensg_grouped['hashed_id'].apply(lambda x: x[0])

# create the same primary_id column for the entries without ENSG ids 
pids_isna['primary_id'] = pids_isna['hashed_id'].to_list()

In [71]:
# now we can explode the hashed ids again and concatenate the ones without ENSG
merged = ensg_grouped.explode('hashed_id').merge(protein_ids[['Protein IDs', 'hashed_id']], how='left')
final_table = pd.concat([merged, pids_isna]).reset_index(drop=True).copy()

In [72]:
final_table

,ensg_ids,hashed_id,primary_id,Protein IDs
0,ENSG00000001167,9ed84952e21a83f98491268a8aa587f0b6b0a0ddc23013...,9ed84952e21a83f98491268a8aa587f0b6b0a0ddc23013...,P23511-2;P23511
1,ENSG00000001497,762329dff28e53d7212fd592f78cb2373152ad22292d01...,762329dff28e53d7212fd592f78cb2373152ad22292d01...,Q9Y4W2-3;Q9Y4W2-2;Q9Y4W2;Q9Y4W2-4
2,ENSG00000001561,c2e387d06c652b2519c95f6946ce48989f528ac7b2e117...,c2e387d06c652b2519c95f6946ce48989f528ac7b2e117...,Q9Y6X5
3,ENSG00000002330,3559314b1011c29e71540043d6e8404cb5a8f1b91b4afb...,3559314b1011c29e71540043d6e8404cb5a8f1b91b4afb...,F5GYS3;A8MXU7;Q92934
4,ENSG00000002549,d0f06c59090de8ce9c6438b465043879eea4b17c0bec26...,d0f06c59090de8ce9c6438b465043879eea4b17c0bec26...,P28838-2;P28838;H0Y9Q1;H0Y983
...,...,...,...,...
3655,None,4fbc68ca0c82cf138f203cb5a20191b4e9fdda107a7586...,4fbc68ca0c82cf138f203cb5a20191b4e9fdda107a7586...,Q58FF8
3656,None,4f5e1cbd13a39caf000337fdf9d09d03cf6933d3f15e9f...,4f5e1cbd13a39caf000337fdf9d09d03cf6933d3f15e9f...,Q6DRA6;Q6DN03
3657,None,48342411580b95ab5e9ba2767122dd20743ae0860868f9...,48342411580b95ab5e9ba2767122dd20743ae0860868f9...,Q9HB66
3658,None,14222158621d1ed21a0c8558617c069125e1aac5fbabcc...,14222158621d1ed21a0c8558617c069125e1aac5fbabcc...,Q9UN81


The final_table now has a clear primary_id that details which protein IDs should be consolidated. 
However, as explained by the Confluence page, these primary_ids are contained only to this specific dataset.
There needs to be a system, preferably on the OC database, to keep track of the primary IDs and which ensg_ids are represented. 

Finally, the generated table can be used to consolidate protein groups and sum up LFQ intensities in the proteingroups.txt file. This should be done prior to imputations. Also, the ENSG mapping can be used for PPI clustering for consistent results.

In [62]:
def proteingroups_to_ensgs(proteingroup, ensembl_uniprot):
    """
    Convert proteingroup Uniprot IDs to a sorted ENSG string
    """
    ensembl_uniprot = ensembl_uniprot.copy() 

    # split proteingroup string into a list of uniques
    pg_list = proteingroup.split(';')
    pg_list = list(set([x.split('-')[0] for x in pg_list]))


    ensgs = []
    for pg in pg_list:
        selection = ensembl_uniprot[ensembl_uniprot['uniprot_id']==pg]
        if selection.shape[0] > 0:
            ensgs.append(selection.iloc[0].ensg_id)
    
    # if there are no ensgs return None
    if len(ensgs) == 0:
        return None
    elif len(ensgs) == 1:
        return ensgs[0]
    ensgs = list(set(ensgs))
    ensgs.sort()
    ensg_str = ';'.join(ensgs)

    return ensg_str


In [9]:
a = ensembl_uniprot[ensembl_uniprot['uniprot_id']=='H0Y394']
a.iloc[0].ensg_id

'ENSG00000115677'